In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.random.seed(601)

import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [3]:
# 날짜 처리
data = pd.read_csv('data/201901-202003.csv')
#data = data.fillna('')

In [4]:
data

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6


In [5]:
data.drop(['CARD_CCG_NM', 'HOM_CCG_NM'   , 'AGE', 'SEX_CTGO_CD', 'FLC'], axis=1, inplace=True)
data

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,CSTMR_CNT,AMT,CNT
0,201901,강원,건강보조식품 소매업,강원,4,311200,4
1,201901,강원,건강보조식품 소매업,강원,7,1374500,8
2,201901,강원,건강보조식품 소매업,강원,6,818700,6
3,201901,강원,건강보조식품 소매업,강원,4,1717000,5
4,201901,강원,건강보조식품 소매업,강원,3,1047300,3
...,...,...,...,...,...,...,...
24697787,202003,충북,휴양콘도 운영업,충북,3,43300,4
24697788,202003,충북,휴양콘도 운영업,충북,3,35000,3
24697789,202003,충북,휴양콘도 운영업,충북,4,188000,6
24697790,202003,충북,휴양콘도 운영업,충북,4,99000,6


In [6]:
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)
data

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,CSTMR_CNT,AMT,CNT,year,month
0,강원,건강보조식품 소매업,강원,4,311200,4,2019,1
1,강원,건강보조식품 소매업,강원,7,1374500,8,2019,1
2,강원,건강보조식품 소매업,강원,6,818700,6,2019,1
3,강원,건강보조식품 소매업,강원,4,1717000,5,2019,1
4,강원,건강보조식품 소매업,강원,3,1047300,3,2019,1
...,...,...,...,...,...,...,...,...
24697787,충북,휴양콘도 운영업,충북,3,43300,4,2020,3
24697788,충북,휴양콘도 운영업,충북,3,35000,3,2020,3
24697789,충북,휴양콘도 운영업,충북,4,188000,6,2020,3
24697790,충북,휴양콘도 운영업,충북,4,99000,6,2020,3


In [7]:
# 데이터 정제
df = data.copy()
#df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'   , 'AGE', 'SEX_CTGO_CD', 'FLC'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [8]:
df

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,2019,1,1058,147831746,1105
1,강원,건강보조식품 소매업,강원,2019,2,1118,146166571,1150
2,강원,건강보조식품 소매업,강원,2019,3,965,109439025,1020
3,강원,건강보조식품 소매업,강원,2019,4,933,105208473,989
4,강원,건강보조식품 소매업,강원,2019,5,1141,146659738,1191
...,...,...,...,...,...,...,...,...
106381,충북,휴양콘도 운영업,충북,2019,11,358,19681520,535
106382,충북,휴양콘도 운영업,충북,2019,12,433,23808100,612
106383,충북,휴양콘도 운영업,충북,2020,1,603,34335376,903
106384,충북,휴양콘도 운영업,충북,2020,2,230,10002800,315


In [9]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

LabelEncoder()

LabelEncoder()

LabelEncoder()

In [10]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [11]:
k = int(len(x)*0.9)

In [12]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [13]:
import lightgbm as lgb

In [14]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [15]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [16]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.98123
[200]	valid_0's rmse: 1.69703
[300]	valid_0's rmse: 1.53514
[400]	valid_0's rmse: 1.41936
[500]	valid_0's rmse: 1.32954
[600]	valid_0's rmse: 1.2741
[700]	valid_0's rmse: 1.22564
[800]	valid_0's rmse: 1.17586
[900]	valid_0's rmse: 1.14232
[1000]	valid_0's rmse: 1.11165
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.11165


In [17]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
#AGEs          = df_num['AGE'].unique()
#SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
#FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            #for AGE in AGEs:
                #for SEX_CTGO_CD in SEX_CTGO_CDs:
                    #for FLC in FLCs:
            for year in years:
                for month in months:
                    temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, year, month]) # , AGE, SEX_CTGO_CD, FLC
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

In [18]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,year,month
0,0,0,0,2020,4
1,0,0,0,2020,7
2,0,0,1,2020,4
3,0,0,1,2020,7
4,0,0,2,2020,4
...,...,...,...,...,...
23693,16,30,10,2020,7
23694,16,30,12,2020,4
23695,16,30,12,2020,7
23696,16,30,14,2020,4


In [19]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [20]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [22]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.899550e+07
1,202004,강원,골프장 운영업,9.146547e+08
2,202004,강원,과실 및 채소 소매업,5.242347e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,2.319548e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.620717e+06
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,3.553439e+08
1390,202007,충북,한식 음식점업,1.715678e+10
1391,202007,충북,호텔업,4.973509e+07


In [25]:
dfall = pd.read_csv('data/201901-202003.csv')
dfall

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6


In [26]:
dfall.drop(['CARD_CCG_NM', 'HOM_CCG_NM'   , 'AGE', 'SEX_CTGO_CD', 'FLC', 'HOM_SIDO_NM','CNT','CSTMR_CNT'], axis=1, inplace=True)
dfall

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,311200
1,201901,강원,건강보조식품 소매업,1374500
2,201901,강원,건강보조식품 소매업,818700
3,201901,강원,건강보조식품 소매업,1717000
4,201901,강원,건강보조식품 소매업,1047300
...,...,...,...,...
24697787,202003,충북,휴양콘도 운영업,43300
24697788,202003,충북,휴양콘도 운영업,35000
24697789,202003,충북,휴양콘도 운영업,188000
24697790,202003,충북,휴양콘도 운영업,99000


In [27]:
columns = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']
dfall = dfall.groupby(columns).sum().reset_index(drop=False)
dfall

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
9433,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299
9434,202003,충북,한식 음식점업,16152482704
9435,202003,충북,호텔업,15248550
9436,202003,충북,화장품 및 방향제 소매업,428881434


In [28]:
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM']
dfall = dfall.groupby(columns).mean().reset_index(drop=False)
dfall

,CARD_SIDO_NM,STD_CLSS_NM,REG_YYMM,AMT
0,강원,건강보조식품 소매업,201925.6,1.213883e+08
1,강원,골프장 운영업,201925.6,3.419392e+09
2,강원,과실 및 채소 소매업,201925.6,1.111699e+09
3,강원,관광 민예품 및 선물용품 소매업,201925.6,3.048080e+07
4,강원,그외 기타 분류안된 오락관련 서비스업,201905.0,1.770000e+05
...,...,...,...,...
645,충북,피자 햄버거 샌드위치 및 유사 음식점업,201925.6,1.419611e+09
646,충북,한식 음식점업,201925.6,2.180969e+10
647,충북,호텔업,201925.6,3.539429e+07
648,충북,화장품 및 방향제 소매업,201925.6,6.116993e+08


In [29]:
dfall.drop(['REG_YYMM'], axis=1, inplace=True)
dfall

,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,강원,건강보조식품 소매업,1.213883e+08
1,강원,골프장 운영업,3.419392e+09
2,강원,과실 및 채소 소매업,1.111699e+09
3,강원,관광 민예품 및 선물용품 소매업,3.048080e+07
4,강원,그외 기타 분류안된 오락관련 서비스업,1.770000e+05
...,...,...,...
645,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.419611e+09
646,충북,한식 음식점업,2.180969e+10
647,충북,호텔업,3.539429e+07
648,충북,화장품 및 방향제 소매업,6.116993e+08


In [30]:
df04a = dfall.copy()
df04a['REG_YYMM'] = 202004
df07a = dfall.copy()
df07a['REG_YYMM'] = 202007
tempa = pd.concat([df04a, df07a])
tempa

,CARD_SIDO_NM,STD_CLSS_NM,AMT,REG_YYMM
0,강원,건강보조식품 소매업,1.213883e+08,202004
1,강원,골프장 운영업,3.419392e+09,202004
2,강원,과실 및 채소 소매업,1.111699e+09,202004
3,강원,관광 민예품 및 선물용품 소매업,3.048080e+07,202004
4,강원,그외 기타 분류안된 오락관련 서비스업,1.770000e+05,202004
...,...,...,...,...
645,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.419611e+09,202007
646,충북,한식 음식점업,2.180969e+10,202007
647,충북,호텔업,3.539429e+07,202007
648,충북,화장품 및 방향제 소매업,6.116993e+08,202007


In [31]:
suba = pd.read_csv('data/submission.csv', index_col=0)
suba = suba.drop(['AMT'], axis=1)
suba = suba.merge(tempa, left_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
suba.index.name = 'id'
suba

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.213883e+08
1,202004,강원,골프장 운영업,3.419392e+09
2,202004,강원,과실 및 채소 소매업,1.111699e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.048080e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.770000e+05
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.419611e+09
1390,202007,충북,한식 음식점업,2.180969e+10
1391,202007,충북,호텔업,3.539429e+07


In [32]:
idx = suba[suba.AMT.isnull() == True].index
idx

Int64Index([  30,   55,   71,   96,  112,  137,  153,  168,  178,  194,  204,
             209,  217,  235,  245,  250,  258,  260,  261,  276,  281,  286,
             291,  332,  368,  381,  383,  384,  392,  397,  399,  404,  407,
             409,  422,  440,  450,  455,  466,  491,  522,  537,  547,  589,
             629,  660,  686,  727,  752,  768,  793,  809,  834,  850,  865,
             875,  891,  901,  906,  914,  932,  942,  947,  955,  957,  958,
             973,  978,  983,  988, 1029, 1065, 1078, 1080, 1081, 1089, 1094,
            1096, 1101, 1104, 1106, 1119, 1137, 1147, 1152, 1163, 1188, 1219,
            1234, 1244, 1286, 1326, 1357, 1383],
           dtype='int64', name='id')

In [33]:
submission.loc[idx, 'AMT'] = 1.0

In [35]:
submission.to_csv('20200707_3.csv', encoding='utf-8-sig')
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.899550e+07
1,202004,강원,골프장 운영업,9.146547e+08
2,202004,강원,과실 및 채소 소매업,5.242347e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,2.319548e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.620717e+06
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,3.553439e+08
1390,202007,충북,한식 음식점업,1.715678e+10
1391,202007,충북,호텔업,4.973509e+07
